# Re-creating R. L. Draper's *top spin*

By [Allison Parrish](https://www.decontextualize.com/)

This notebook shows how to recreate R. L. Draper's concrete poem *top spin*, as reproduced in [How to read a concrete poem](http://www.futureanachronism.com/digitalpaper/HowToReadAConcretePoem.pdf), with Python ranges, `for` loops, and string indexing.

The original looks like this:

    tops
     ops
      ps
       s
       sp
       spi
       spin
       spi
       sp
       s
      ps
     ops
    tops
     ops
      ps
       s
       sp
       spi
       spin
       spi
       sp
       s
      ps
     ops
    tops
    
We're going to write some code that achieves this same effect, but can produce an arbitrary number of repetitions and work on any input string.

Here's my formulation of how *top spin* works. Starting withh a source string, in this case `topspin`, the poem contains successive lines that contain a substring of the source string, either ending with the letter in the middle or starting with the letter in the middle. The particular substring shown starts at an index that increases until it reaches the end of the string, then decreases until it reaches the beginning of the string, and then repeats.

So how to implement this in python?

First, we'll assign our source string to a variable `s`. (Keeping the variable name short because we'll be using it a lot.)

In [1]:
s = "topspin"

To find the index halfway through the string, divide the length of `s` by two:

In [2]:
h = int(len(s) / 2)

In [3]:
h

3

Slicing up to this index gives us the first half of the string:

In [4]:
s[:h]

'top'

And from this index to the end gives us the second half of the string:

In [5]:
s[-h:]

'pin'

So far so good.

## Generating triangles

In order to create the repeating pattern of substrings, we need to generate a list of indices that goes up to the end of the string and then back to the beginning. Ideally, we'd end up with something like this list:

    [0, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1]
    
... which goes from zero up to the last index of the source string (in this case, `6`) and then back down, stopping short before zero so the entire sequence can be repeated. We'll call this a *triangle range*, since if you visualize it, it looks like a sequence of triangles:

        *       *
       * *     * *
      *   *   *   *
     *     * *     *
    *       *       * (etc.)

The `range()` function in Python can produce the ascending part of this list:

In [6]:
for i in range(len(s)):
    print(i)

0
1
2
3
4
5
6


To go back down, you can use the `reversed()` function on the result of `range()`:

In [7]:
for i in reversed(range(len(s))):
    print(i)

6
5
4
3
2
1
0


If the `range()` and `reversed()` functions returned *lists*, then we could just concatenate them together to get the up-and-down range we want. They're not lists, however; they're [iterable objects](https://treyhunner.com/2018/02/python-range-is-not-an-iterator/) that work as the thing being looped over in a `for` loop. The cleanest way to concatenate these is with the `chain` function from `itertools`, which chains together iterable objects (whether lists, generators, sequences, etc.):

In [8]:
from itertools import chain

In [9]:
a = range(4)
b = ["a", "b", "c", "d"]
for item in chain(a, b):
    print(item)

0
1
2
3
a
b
c
d


To create our up-and-down range, then, should be as easy as:

In [10]:
for item in chain(range(len(s)), reversed(range(len(s)))):
    print(item)

0
1
2
3
4
5
6
6
5
4
3
2
1
0


That's not quite right, since we're repeating both the highest index and the lowest index. By adjusting the start and end parameters of the `range()` function, we can work around this:

In [11]:
for i in chain(range(len(s)-1), reversed(range(1, len(s)))):
    print(i)

0
1
2
3
4
5
6
5
4
3
2
1


In the following cell, I define a function that returns the chained iterables to create a triangle range with a given upper limit. We'll use this in the rest of the notebook:

In [12]:
def trirange(n):
    return chain(range(n-1), reversed(range(1, n)))
list(trirange(4))

[0, 1, 2, 3, 2, 1]

## Approximating *top spin*

Now that we can work up and down the range, we can start making things that are vaguely *top spin*-like. In this first example, I simply print out the letter at the corresponding index from a triangle range going up to the length of the string:

In [13]:
for i in trirange(len(s)):
    print(s[i])

t
o
p
s
p
i
n
i
p
s
p
o


The multiplication operator `*` works on strings as well as numbers. When you use it with a string *t* on the left and a number *n* on the right, it produces a new string that has *n* copes of *t* in a row:

In [14]:
"x" * 10

'xxxxxxxxxx'

In [15]:
"howdy" * 20

'howdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdyhowdy'

Taking advantage of this, we can produce a variation on *top spin* that has each letter at its index on subsequent lines, following a triangle range:

In [16]:
for i in trirange(len(s)):
    print((" " * i) + s[i])

t
 o
  p
   s
    p
     i
      n
     i
    p
   s
  p
 o


Using an outer `for` loop to repeat this several times:

In [17]:
for count in range(3):
    for i in trirange(len(s)):
        print((" " * i) + s[i])

t
 o
  p
   s
    p
     i
      n
     i
    p
   s
  p
 o
t
 o
  p
   s
    p
     i
      n
     i
    p
   s
  p
 o
t
 o
  p
   s
    p
     i
      n
     i
    p
   s
  p
 o


We're nearly there! Now all we need is to write code that follows the rules we originally set out: following a triangle range, if the index is before the middle of the string, display the substring up to the middle (plus the middle character); otherwise, display the substring from the middle up to the end (inclusive). Here's code that implements that. (Remember that we defined `h` above to be the middle index of the string.

In [18]:
for i in trirange(len(s)):
    if i < h:
        print(s[i:h+1].rjust(h+1)) # +1 to include the middle character
    else:
        print((" " * h) + s[h:i+1])

tops
 ops
  ps
   s
   sp
   spi
   spin
   spi
   sp
   s
  ps
 ops


Note the use of [`.rjust()`](https://docs.python.org/3/library/stdtypes.html?highlight=rjust#str.rjust) to justify the string to the right, given a particular length.

## Generalizing *top spin*

Just to make things cleaner and to make it easier to experiment with different inputs and outputs, I define a function below to create one "spin" of *top spin*:

In [19]:
def spin(s):
    h = int(len(s) / 2)
    out = []
    for i in trirange(len(s)):
        if i < h:
            out.append(s[i:h+1].rjust(h+1)) # +1 to include the middle character
        else:
            out.append((" " * h) + s[h:i+1])
    return "\n".join(out)

Now we can "spin" multiple times:

In [20]:
for i in range(4):
    print(spin("topspin"))

tops
 ops
  ps
   s
   sp
   spi
   spin
   spi
   sp
   s
  ps
 ops
tops
 ops
  ps
   s
   sp
   spi
   spin
   spi
   sp
   s
  ps
 ops
tops
 ops
  ps
   s
   sp
   spi
   spin
   spi
   sp
   s
  ps
 ops
tops
 ops
  ps
   s
   sp
   spi
   spin
   spi
   sp
   s
  ps
 ops


And spin arbitrary strings:

In [21]:
for i in range(2):
    print(spin("allisonparrish"))

allisonp
 llisonp
  lisonp
   isonp
    sonp
     onp
      np
       p
       pa
       par
       parr
       parri
       parris
       parrish
       parris
       parri
       parr
       par
       pa
       p
      np
     onp
    sonp
   isonp
  lisonp
 llisonp
allisonp
 llisonp
  lisonp
   isonp
    sonp
     onp
      np
       p
       pa
       par
       parr
       parri
       parris
       parrish
       parris
       parri
       parr
       par
       pa
       p
      np
     onp
    sonp
   isonp
  lisonp
 llisonp


## Experimental *top spin*s

Here are a few variations on *top spin* for you to work through and play with.

First off, a version that prints the negative space of the triangle range:

In [22]:
def negativespace(s):
    out = []
    for i in trirange(len(s)):
        missed = s[:i] + " " + s[i+1:]
        out.append(missed)
    return "\n".join(out)

In [23]:
for i in range(2):
    print(negativespace("alphabet"))

 lphabet
a phabet
al habet
alp abet
alph bet
alpha et
alphab t
alphabe 
alphab t
alpha et
alph bet
alp abet
al habet
a phabet
 lphabet
a phabet
al habet
alp abet
alph bet
alpha et
alphab t
alphabe 
alphab t
alpha et
alph bet
alp abet
al habet
a phabet


Next, a version that prints a zigzag "stripe" from the string:

In [24]:
def zigzag(s, chunklen=2):
    out = []
    for i in trirange(len(s)-(chunklen-1)):
        out.append(" " * i + s[i:i+chunklen])
    return "\n".join(out)

In [25]:
for i in range(4):
    print(zigzag("zigzag", 3))

zig
 igz
  gza
   zag
  gza
 igz
zig
 igz
  gza
   zag
  gza
 igz
zig
 igz
  gza
   zag
  gza
 igz
zig
 igz
  gza
   zag
  gza
 igz


And finally, a new range function that returns an range of indices that follow a sine wave (well, actually an inverted *cosine* wave) instead of a triangle, smoothly ramping up and back down:

In [26]:
import math
def sinerange(n, steps=10):
    for i in range(steps):
        theta = (math.tau / steps) * i
        yield int(n/2 * (-1*math.cos(theta))) + int(n/2)

The second parameter (`steps`) sets the "smoothness" of the wave:

In [27]:
list(sinerange(10, 25))

[0, 1, 1, 2, 3, 4, 5, 5, 7, 8, 9, 9, 9, 9, 9, 9, 8, 7, 5, 5, 4, 3, 2, 1, 1]

Here's the single-letter variant of *top spin* produced with this range function:

In [28]:
for i in range(2):
    for i in sinerange(len(s), steps=16):
        print(" " * i + s[i])

t
t
 o
  p
   s
    p
     i
      n
      n
      n
     i
    p
   s
  p
 o
t
t
t
 o
  p
   s
    p
     i
      n
      n
      n
     i
    p
   s
  p
 o
t


Combining the zigzag stripe with the sine range:

In [29]:
def sinezigzag(s, chunklen=2, steps=16):
    out = []
    for i in sinerange(len(s)-(chunklen-1), steps):
        out.append(" " * i + s[i:i+chunklen])
    return "\n".join(out)

In [30]:
for i in range(3):
    print(sinezigzag("trigonometry!", 5, 24))

trigo
trigo
 rigon
 rigon
  igono
   gonom
    onome
     nomet
      ometr
       metry
       metry
        etry!
        etry!
        etry!
       metry
       metry
      ometr
     nomet
    onome
   gonom
  igono
 rigon
 rigon
trigo
trigo
trigo
 rigon
 rigon
  igono
   gonom
    onome
     nomet
      ometr
       metry
       metry
        etry!
        etry!
        etry!
       metry
       metry
      ometr
     nomet
    onome
   gonom
  igono
 rigon
 rigon
trigo
trigo
trigo
 rigon
 rigon
  igono
   gonom
    onome
     nomet
      ometr
       metry
       metry
        etry!
        etry!
        etry!
       metry
       metry
      ometr
     nomet
    onome
   gonom
  igono
 rigon
 rigon
trigo
